# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

# Описание данных

`children` — количество детей в семье  
`days_employed` — общий трудовой стаж в днях  
`dob_years` — возраст клиента в годах  
`education` — уровень образования клиента  
`education_id` — идентификатор уровня образования  
`family_status` — семейное положение  
`family_status_id` — идентификатор семейного положения  
`gender` — пол клиента  
`income_type` — тип занятости  
`debt` — имел ли задолженность по возврату кредитов  
`total_income` — ежемесячный доход  
`purpose`— цель получения кредита

**План работ:**

- Ознакомление с данными
- Предобработка данных
- Выяление зависимостей
- Выводы по поставленным вопросам

## Шаг 1. Ознакомление с данными
Загрузим необходимые библиотеки и произведем первичное ознакомление с данными.

In [1]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

In [2]:
m = Mystem()

In [3]:
data = pd.read_csv(r'C:\Users\79119\Desktop\DF\reliability_research\data.csv')

In [4]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**  
В представленной таблице 21524 строки и 12 столбцов, из которых:  
- 5 - типа int;  
- 2 - типа float;  
- 5 - типа object.  

Также в столбцах 'days_employed' и 'total_income' присутствуют пропуски. 

## Шаг 2. Предобработка данных

### Обработка пропусков

Для начала, посчитаем количесство пропущенных значений в каждом из столбцов.

In [6]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Количество пропусков в `days_employed` и `total_income` совпадают. Проверим, одни и теже ли строки с пропусками в `days_employed` и `total_income`. Для этого создадим переменную passes, в которой будут хранится строки с пропусками в столбце `days_employed`

In [7]:
passes = data[data['days_employed'].isna()]

In [8]:
passes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          2174 non-null   int64  
 1   days_employed     0 non-null      float64
 2   dob_years         2174 non-null   int64  
 3   education         2174 non-null   object 
 4   education_id      2174 non-null   int64  
 5   family_status     2174 non-null   object 
 6   family_status_id  2174 non-null   int64  
 7   gender            2174 non-null   object 
 8   income_type       2174 non-null   object 
 9   debt              2174 non-null   int64  
 10  total_income      0 non-null      float64
 11  purpose           2174 non-null   object 
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


**Вывод**  
Как видно из общей информации по таблице passes, строки с пропусками в столбцах с общим трудовым стажем и ежемесячным доходом совпадают. Соответчтвенно там, где не указан стаж работы, не указан доход. 

### Замена типа данных

Для сохранения 10% данных, заменим пропуски в столбцах на медианное значение в зависимости от категории занятости.

In [9]:
data['total_income'] = data['total_income'].fillna(data.groupby('income_type')['total_income'].transform('median'))
data['total_income'] = data['total_income'].fillna(data['total_income'].median())

In [10]:
data['days_employed'] = data['days_employed'].fillna(data.groupby('income_type')['days_employed'].transform('median'))
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median())

Проверим, что пропусков больше нет

In [11]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Приведем столбцы со стажем и доходом к целочисленные значениям (тип int)

In [12]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

Проверим, что преобразование проведено корректно.

In [13]:
data.dtypes

children             int64
days_employed        int32
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                 int64
total_income         int32
purpose             object
dtype: object

Проверим количественные признаки на наличие аномалий

In [14]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.538908,63550.801812,43.293380,0.817236,0.972544,0.080883,1.652248e+05
std,1.381587,141150.053986,12.574584,0.548138,1.420324,0.272661,9.804367e+04
min,-1.000000,-18388.000000,0.000000,0.000000,0.000000,0.000000,2.066700e+04
25%,0.000000,-2570.000000,33.000000,1.000000,0.000000,0.000000,1.077980e+05
50%,0.000000,-1355.000000,42.000000,1.000000,0.000000,0.000000,1.425940e+05
75%,1.000000,-316.000000,53.000000,1.000000,1.000000,0.000000,1.955490e+05
max,20.000000,401755.000000,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Выявлены аномальные значения -1 и 20 в столбце с количеством детей, также отрицательные и положительные значения в количестве дней трудового стажа.

In [15]:
data.groupby('children')['income_type'].count()

children
-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: income_type, dtype: int64

В связи с тем, что объем данных с аномальным значение в столбце количества детей, составляет менее 1%, эти строки решено удалить.

In [16]:
data = data.drop(data[data['children'] == -1].index)
data = data.drop(data[data['children'] == 20].index)

Проверим, что значения удалены корректно

In [17]:
data.groupby('children')['income_type'].count()

children
0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: income_type, dtype: int64

Проверим, есть ли какая-то закономерность между странными значениями в трудовом стаже и типом занятости. Посмотрим минимальное и максимальное значение столбца `days_employed` в разбивке по типам занятости.

In [18]:
data.groupby('income_type')['days_employed'].min()

income_type
безработный        337524
в декрете           -3296
госслужащий        -15193
компаньон          -17615
пенсионер          328728
предприниматель      -520
сотрудник          -18388
студент              -578
Name: days_employed, dtype: int32

In [19]:
data.groupby('income_type')['days_employed'].max()

income_type
безработный        395302
в декрете           -3296
госслужащий           -39
компаньон             -30
пенсионер          401755
предприниматель      -520
сотрудник             -24
студент              -578
Name: days_employed, dtype: int32

Столбец `income_type` указан в количестве дней, поэтому при делении отрицательных значений на 365  получается какое-то логичное количество лет. Но положительные значения слишком велики и присутствуют только в двух типах занятости: безработные и пенсионеры.

In [20]:
data['gender'].value_counts()

F      14154
M       7247
XNA        1
Name: gender, dtype: int64

В столбце, который отображает пол заемщика, есть одно нестандартное значение(возможно какая-то небинарная личность)

**Вывод**  
1.Заменили пропуски в столбцах 'days_employed' и 'total_income' на медианное значение по группам занятости, чтобы не терять эти данные.   
2.Округлила до целых чисел столбцы 'days_employed' и 'total_income'.  
3.Выявлены аномальные значения в столбце `children`, но в связи с тем, что природу их установить не удалось, а количество таких данных составляет менее 1% было принято решения данные строки удалить.  
4.Обнаружила закономерность: в столбце 'days_employed' значения указаны в днях, если с отрицаательными значениями при делении на 365 все более или менее логично, то положительные значения только у категорий пенсионеры и безработный, и эти положительные значения при делении 365 получаются запредельно большими. Но в связи с тем, что в выводах значения этого столца не участвуют, никаких изменеий с ним не производились, кроме округления до целых чисел.  
5. Также в столбце `gender` обнаружено одно нестандартное значение. Т.к. это никак не повлияет на дальнейших ход исследования, данное значение оставлено без изменений.

### Обработка дубликатов

Посчитаем, сколько же явных дубликатов в таблицу

In [21]:
data.duplicated().sum()

54

Проверим уникальные значения в столбце 'education'.

In [22]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Как видно, в таблице много повтров в связи с тем, что разнится написание. Поэтому приведем все значения к нижнему ркгистру и избавимся от дубликатов.

In [23]:
data['education'] = data['education'].str.lower()
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [24]:
data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()

0

Проверим, что все дубликаты удалены.

In [25]:
data.duplicated().sum()

0

**Вывод**

В процессе исследования было выявлено 54 дубликата и неявные дубликаты в столбце `education`, которые образовались из-за того, что значения были заполнены в разном регистре. После приведений значений в данном столбце к нижнему регистру, все дублирующиеся значения были удалены.

### Лемматизация

Чтобы выявить основные цели получения кредита, провизведем лемматизацию и подчет наиболее часто встречающихся лемм для дальнейшей категоризации столбца `purpose`

In [26]:
text = " ".join(data['purpose'])
lemmas = m.lemmatize(text)
display(Counter(lemmas))

Counter({'покупка': 5865,
         ' ': 54708,
         'жилье': 4437,
         'приобретение': 459,
         'автомобиль': 4279,
         'дополнительный': 899,
         'образование': 3988,
         'сыграть': 760,
         'свадьба': 2313,
         'операция': 2587,
         'с': 2900,
         'на': 2207,
         'проведение': 763,
         'для': 1286,
         'семья': 637,
         'недвижимость': 6314,
         'коммерческий': 1303,
         'жилой': 1222,
         'строительство': 1867,
         'собственный': 628,
         'подержать': 842,
         'свой': 2219,
         'со': 623,
         'заниматься': 904,
         'сделка': 936,
         'получение': 1309,
         'высокий': 1367,
         'подержанный': 110,
         'профильный': 432,
         'сдача': 649,
         'ремонт': 604,
         '\n': 1})

**Вывод**

Лемматизация и подсчет лемм была произведена, для того чтобы выявить наиболее популярные слова. А это в дальнейщем поможет разбить эти значения на категории. В данном случае выявлены лидеры: жилье, недвижимость, автомобиль, свадьба, образование

### Категоризация данных

Для удобства создадим солццы `purpose_category` с категориями целей получения кредита: покупка недвтжимости, проведение свадьбы, покупка автомобиля и получение образования и `total_income_category` с категориями уровня ежемесячного дохода: низкий от 0 до 100000, средний от 100000 до 500000 и высокий от 500000 и выше.

In [27]:
def scroll (purpose):
    if  'недвижим' in purpose or 'жиль' in purpose:
        return 'покупка недвижимости'
 
    elif 'свадьб'in purpose:
        return 'проведение свадьбы'
 
    elif 'автомоб'in purpose:
        return 'покупка автомобиля'
    return 'получение образования'

data['purpose_category'] = data['purpose'].apply(scroll)
data['purpose_category'].unique()
# для категоризации применена функция одной строки

array(['покупка недвижимости', 'покупка автомобиля',
       'получение образования', 'проведение свадьбы'], dtype=object)

In [28]:
data['total_income'].agg(['min', 'max'])

min      20667
max    2265604
Name: total_income, dtype: int64

In [29]:
data.query('0 < total_income < 100000')['children'].count()

4444

In [30]:
data.query('101000 <= total_income < 500000')['children'].count()

16542

In [31]:
data.query('501000 <= total_income <= 2265604')['children'].count()

222

In [32]:
def counter(income):
    if income <= 100000:
        return 'низкий доход'
    if income <= 500000:
        return 'средний доход'
    return 'высокий доход'
data['total_income_category'] = data['total_income'].apply(counter)
data['total_income_category'].unique()

array(['средний доход', 'низкий доход', 'высокий доход'], dtype=object)

**Вывод**

Добавлено 2 новых столбца 'purpose_category' и 'total_income_category' с категориями данных по целям кредита и доходу для проверки зависимости между семейным положением и возвратом кредита в срок, и зависимости между уровнем дохода и возврата кредита.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [33]:
pivot_table_children = data.pivot_table(index= 'children', values='debt', aggfunc='mean')
pivot_table_children

,debt
children,
0,0.075438
1,0.092346
2,0.094542
3,0.081818
4,0.097561
5,0.000000


**Вывод**

При наличие детей в семье растет риск невыплаты по кредиту, вероятней всего это связано с увеличением финансовой нагрузки на каждого ребенка и соотвественно снижением платежеспособности по кредиту. Там где 5 детей показетель можно не принимать к расчету, т.к. таких клиентов вего 9 человек в таблице.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [34]:
family_status_table = data.pivot_table(index= 'family_status', values='debt', aggfunc='mean').reset_index()
family_status_table

,family_status,debt
0,Не женат / не замужем,0.097639
1,в разводе,0.070648
2,вдовец / вдова,0.066246
3,гражданский брак,0.093130
4,женат / замужем,0.075606


**Вывод**

Наибольшая доля невозврата у категории граждан без каких-либо официально оформленных отношений (не женат/не замужем и гражданский брак). Нет овественноси перед семьей = нет отвественности перед банком..?

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [35]:
total_income_table = data.pivot_table(index= 'total_income_category', values='debt', aggfunc='mean')
total_income_table

,debt
total_income_category,
высокий доход,0.063063
низкий доход,0.079658
средний доход,0.081848


In [36]:
data.groupby('total_income_category')['total_income'].count()

total_income_category
высокий доход      222
низкий доход      4444
средний доход    16665
Name: total_income, dtype: int64

**Вывод**

Самая высокая доля невозврата у категории заемщиков с высоким уровнем дохода. Возможно часть этих клиентов имеют свое дело и высокую кредитную нагрузку на его развитие, а возможно это опечатка при заполнении анкеты и , к примеру, был проставлен лишний ноль. Таких клиентов примерно 1%. Далее с небольшой разницей в доле невозврата идут низкий и средний уровень дохода.

- Как разные цели кредита влияют на его возврат в срок?

In [37]:
purpose_table = data.pivot_table(index='purpose_category', values='debt', aggfunc='mean')
purpose_table

,debt
purpose_category,
покупка автомобиля,0.093480
покупка недвижимости,0.072551
получение образования,0.092528
проведение свадьбы,0.079118


In [38]:
data['purpose_category'].value_counts()

покупка недвижимости     10751
покупка автомобиля        4279
получение образования     3988
проведение свадьбы        2313
Name: purpose_category, dtype: int64

**Вывод**

Кредит на образование и авто являются лидерами по невозврату. Количество клиентов по этим категориям тоже примерно схоже.

## Шаг 4. Общий вывод

На мой взгляд, основными критериями для оценки возвратности по кредиту нужно брать доход и количество детей, т.к. это напрямую связано с платежеспособностью клиента. Семейное положение косвенно может дать представление, если принять в расчет, что отвеноность перед семьей = отвественность в общем и перед банком в частности. Касаемо целей, чем крупнее цель - тем более отвественный подход к ее реализации, но в нашем конкретном примере цель расходов свадьба на мой взгляд не самая крупная, но тут возможно предположение, описанное ранее отвественность перед семьей = отвественност перед банком.